In [1]:
from imports import *
from PreProcessor import *

_PreProcessor = PreProcessor()


c:\Users\basse\anaconda3\envs\cuda_env\lib\site-packages\tree_sitter\__init__.py:36: FutureWarning: Language.build_library is deprecated. Use the new bindings instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)
c:\Users\basse\anaconda3\envs\cuda_env\lib\site-packages\tree_sitter\__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


In [2]:
function_names_train,function_segments_train = _PreProcessor.setupDeepLearning("./dataset/teiid_dataset/train_CC",'CC','train')
descriptions_train, summaries_train = _PreProcessor.setupDeepLearning("./dataset/teiid_dataset/train_UC",'UC','train')

function_names_test,function_segments_test = _PreProcessor.setupDeepLearning("./dataset/teiid_dataset/test_CC",'CC','test')
descriptions_test, summaries_test = _PreProcessor.setupDeepLearning("./dataset/teiid_dataset/test_UC",'UC','test')

In [3]:
_PreProcessor.setUpUnknown(function_names_train,function_segments_train,'CC')
_PreProcessor.setUpUnknown(descriptions_train,summaries_train,'UC')

_PreProcessor.setUpUnknown(function_names_test,function_segments_test,'CC')
_PreProcessor.setUpUnknown(descriptions_test,summaries_test,'UC')

In [4]:
with open('./pickles/DeepLearning/function_names_train.pkl', 'wb') as f:
       pickle.dump(function_names_train, f)
with open('./pickles/DeepLearning/function_segments_train.pkl', 'wb') as f:
        pickle.dump(function_segments_train, f)
with open('./pickles/DeepLearning/descriptions_train.pkl', 'wb') as f:
        pickle.dump(descriptions_train, f)
with open('./pickles/DeepLearning/summaries_train.pkl', 'wb') as f:
        pickle.dump(summaries_train, f)

with open('./pickles/DeepLearning/Vocab.pkl', 'wb') as f:
        pickle.dump(_PreProcessor.Vocab, f)

with open('./pickles/DeepLearning/function_names_test.pkl', 'wb') as f:
       pickle.dump(function_names_test, f)
with open('./pickles/DeepLearning/function_segments_test.pkl', 'wb') as f:
        pickle.dump(function_segments_test, f)
with open('./pickles/DeepLearning/descriptions_test.pkl', 'wb') as f:
        pickle.dump(descriptions_test, f)
with open('./pickles/DeepLearning/summaries_test.pkl', 'wb') as f:
        pickle.dump(summaries_test, f)

In [5]:
function_names_train = np.load('./pickles/DeepLearning/function_names_train.pkl',allow_pickle=True)
function_segments_train = np.load('./pickles/DeepLearning/function_segments_train.pkl',allow_pickle=True)
descriptions_train = np.load('./pickles/DeepLearning/descriptions_train.pkl',allow_pickle=True)
summaries_train = np.load('./pickles/DeepLearning/summaries_train.pkl',allow_pickle=True)
vocab = np.load('./pickles/DeepLearning/Vocab.pkl',allow_pickle=True)
function_names_test = np.load('./pickles/DeepLearning/function_names_test.pkl',allow_pickle=True)
function_segments_test = np.load('./pickles/DeepLearning/function_segments_test.pkl',allow_pickle=True)
descriptions_test = np.load('./pickles/DeepLearning/descriptions_test.pkl',allow_pickle=True)
summaries_test = np.load('./pickles/DeepLearning/summaries_test.pkl',allow_pickle=True)

### Converting dataset to indices

In [4]:
_PreProcessor.vocabToIndex()
_PreProcessor.dataSetToIndex(function_names_train,function_segments_train,'CC')
_PreProcessor.dataSetToIndex(descriptions_train,summaries_train,'UC')

_PreProcessor.dataSetToIndex(function_names_test,function_segments_test,'CC')
_PreProcessor.dataSetToIndex(descriptions_test,summaries_test,'UC')

In [5]:
x_train,labels = _PreProcessor.setUpLabels(function_names_train,function_segments_train,descriptions_train,summaries_train)


In [6]:
print(x_train[0],labels[0] )

[[tensor([1]), tensor([1, 2]), tensor([3, 4]), tensor([1, 5]), tensor([6, 7, 8]), tensor([ 7,  9, 10, 11]), tensor([12, 13]), tensor([ 6, 14, 15, 13]), tensor([16, 17, 18]), tensor([19, 20, 21, 22]), tensor([23, 24]), tensor([25, 26, 18]), tensor([27, 28, 15, 13]), tensor([221,   5])], [tensor([29, 30, 31, 31, 29, 29, 30, 31, 32, 33, 34, 30, 31, 32,  1,  2, 35, 36,
        35, 33, 34,  1,  2, 35,  5, 36, 35,  5, 37,  5, 36,  5, 36,  5,  1,  2,
         2,  5, 38, 36, 39, 40, 41, 39, 40, 41, 39, 40, 41, 39, 29, 30, 31, 32,
        36,  2, 36,  5, 33, 34, 30, 31, 32, 33, 34, 37,  2, 36,  2, 37,  2, 36,
         5, 36,  2, 14, 13, 42]), tensor([ 1, 43,  5,  2, 35, 37,  5,  1,  5,  1,  5,  2,  5,  1,  5, 44, 45,  2,
         5,  3, 46, 47,  1,  2,  2,  5,  3, 48, 46, 37,  5, 49, 50,  1,  5,  1,
         5, 51, 52,  5, 53, 54, 55, 49, 50,  3, 56, 49, 50, 49, 50,  3, 56, 57,
        58,  1,  2, 59, 56,  2,  5, 46,  5,  2,  5,  3, 56, 37,  5, 46,  1,  2,
        46,  5,  2,  5,  3, 60,  5, 61

In [11]:
class TracibilityLinkDataset(Dataset):

    def __init__(self, x_train, y_train):

        # # getting the max number of funtions in a file along all files
        # #getting the longest name of a function along all functions in all files
        # max_number_function = max(len(_) for _ in x_train_function_names)
        # max_function_name_len = max(len(function) for file in x_train_function_names for function in file)

        # #getting the longest segment of a function along all functions in all files
        # max_function_segment_len = max(len(function) for file in x_train_function_segments for function in file)

        # x_train_function_names_padded = [file + [[]] * (max_number_function - len(file)) for file in x_train_function_names]
        # x_train_function_names_padded = [function + [vocab_size+1] * (max_number_function - len(function)) for file in x_train_function_names_padded for function in file]
        
        # # convert it to a tensor to be used in model
        # self.tensor_x_train_function_names_padded = torch.tensor(x_train_function_names_padded)
        
        self.tensor_x_train=x_train
        self.tensor_y_train = torch.tensor(y_train)


    # len and get item are very important --> used by dataloader
    def __len__(self):
        return len(self.tensor_x_train)

    def __getitem__(self, idx):
        return self.tensor_x_train[idx], self.tensor_y_train[idx]


In [16]:
dataset_train = TracibilityLinkDataset(x_train, labels)
torch.save(dataset_train,'./dataset/teiid_dataset/DeepLearningDataset.pt')